In [1]:
import requests
from bs4 import BeautifulSoup
import openai
import json
#####################
from tokens.key import github_key # import your key
from tokens.key import openai_key # import your key

In [2]:
catogary_lsit_RQ2=[ 
"Domain and architecture agnostic algorithm and data structure optimization" ,
"Domain specific" ,
"Micro-architecture specific optimization" ,
"Enable compiler optimizations" ,
"Faster API call" ,
"Introduce parallelization" ,
"Balancing parallel load" ,
"Improved concurrency control" ,
"Memory management" ,
"fix programming error" ,]


In [3]:
sub_catogary_dict_RQ2 = {
    "Domain and architecture agnostic algorithm and data structure optimization": {
        "score": 17,
        "subcategories": [
            "data structure transformation",
            "lower complexity algorithm - domain and arch agnostic",
            "Caching",
            "Use of fast data strcture container",
        ],
    },
    "Domain specific": {
        "score": 27,
        "subcategories": [
            "Reduce iteration/traversal of data structure",
            "Eliminating unnecessary operation",
            "Avoid unnecessary communication",
            "Batching function calls",
            "lazy housekeeping",
            "code specialization",
            "lower complexity algorithm - domain specific",
        ],
    },
    "Micro-architecture specific optimization": {
        "score": 60,
        "subcategories": [
            "Architecture specific logic modification",
            "Architecture specific fast kernel",
            "reduced precision",
            "strength reduction",
            "Architecture specific fast instruction",
            "Use of data types that reduces computation and memory overhead",
            "Data locality optimization",
        ],
    },
    "Enable compiler optimizations": {
        "score": 27,
        "subcategories": [
            "Function inlining",
            "Enable compile time evaluation",
            "Scalar Replacement",
            "Loop optimization",
        ],
    },
    "Faster API call": {"score": 2, "subcategories": []},
    "Introduce parallelization": {
        "score": 11,
        "subcategories": [
            "OpenMP",
            "oneAPI",
            "Introduce vectorization",
            "pthread-based paralleism",
        ],
    },
    "Balancing parallel load": {
        "score": 17,
        "subcategories": [
            "Explicitly set thread numbers",
            "Sort workload and schedule",
            "Tune task size",
            "Reduce parallelization",
        ],
    },
    "Improved concurrency control": {
        "score": 7,
        "subcategories": [
            "Eliminate unnecessary lock",
            "Efficient lock implementation with atomics",
            "Replace strong consistency with an weaker one",
            "Eliminate unncessary synchronization",
            "Rewrite kernel to make memory access warp-synchronous",
        ],
    },
    "Memory management": {
        "score": 13,
        "subcategories": [
            "Avoid memory leak by memory management",
            "Pre-allocation",
            "Increase memory size for efficiency",
            "Use efficient API",
            "Reference rather than copy",
        ],
    },
    "fix programming error": {"score": 4, "subcategories": []},
    "total": 185,
}


In [4]:
sub_sub_catogary_dict_RQ2 = {
    "Domain and architecture agnostic algorithm and data structure optimization": {
        "count": 17,
        "subcategories": {
            "data structure transformation": 3,
            "lower complexity algorithm - domain and arch agnostic": 1,
            "Caching": 6,
            "Use of fast data strcture container": 7,
        },
    },
    "Domain specific": {
        "count": 27,
        "subcategories": {
            "Reduce iteration/traversal of data structure": 8,
            "Eliminating unnecessary operation": 8,
            "Avoid unnecessary communication": 2,
            "Batching function calls": 3,
            "lazy housekeeping": 1,
            "code specialization": 2,
            "lower complexity algorithm - domain specific": 3,
        },
    },
    "Micro-architecture specific optimization": {
        "count": 60,
        "subcategories": {
            "Architecture specific logic modification": 3,
            "Architecture specific fast kernel": 3,
            "reduced precision": 3,
            "strength reduction": 6,
            "Architecture specific fast instruction": 3,
            "Use of data types that reduces computation and memory overhead": 4,
            "Data locality optimization": {
                "count": 38,
                "sub_subcategories": [
                    "Avoid memory dereference by storing in register",
                    "Reorder memory reference to improve temporal locality",
                    "Reorder memory reference to improve spatial locality",
                    "Improve register/cache utilization via blocking",
                    "Avoid using GPU texture memory",
                    "Reduce GPU-global memory access",
                    "Thread-aware data access",
                    "thread scheduling to improve locality",
                    "Configure HW caches size",
                    "memory pinning",
                    "Memory prefetching",
                    "Async data transfer between host and device",
                    "Data structure optimization for data locality",
                    "Tuning kernel size",
                ],
            },
        },
    },
    "Enable compiler optimizations": {
        "count": 27,
        "subcategories": {
            "Function inlining": 4,
            "Enable compile time evaluation": 5,
            "Scalar Replacement": 1,
            "Loop optimization": {
                "count": 17,
                "sub_subcategories": [
                    "Loop fission",
                    "kernel fusion",
                    "Loop unrolling",
                    "Loop invariant code motion",
                ],
            },
        },
    },
    "Faster API call": {"count": 2, "subcategories": {}},
    "Introduce parallelization": {
        "count": 11,
        "subcategories": {
            "OpenMP": 2,
            "oneAPI": 1,
            "Introduce vectorization": 5,
            "pthread-based paralleism": 3,
        },
    },
    "Balancing parallel load": {
        "count": 17,
        "subcategories": {
            "Explicitly set thread numbers": 4,
            "Sort workload and schedule": 1,
            "Tune task size": 9,
            "Reduce parallelization": 3,
        },
    },
    "Improved concurrency control": {
        "count": 7,
        "subcategories": {
            "Eliminate unnecessary lock": 3,
            "Efficient lock implementation with atomics": 1,
            "Replace strong consistency with an weaker one": 1,
            "Eliminate unncessary synchronization": 1,
            "Rewrite kernel to make memory access warp-synchronous": 1,
        },
    },    "Memory management": {
        "count": 13,
        "subcategories": {
            "Avoid memory leak by memory management": 5,
            "Pre-allocation": 4,
            "Increase memory size for efficiency": 1,
            "Use efficient API": 2,
            "Reference rather than copy": 1,
        },
    },
    "fix programming error": {"count": 4, "subcategories": {}},
}

# Github scraping example 

In [20]:
def get_github_commit_info(commit_url):
    response = requests.get(commit_url)
    if response.status_code != 200:
        print("Error: Unable to fetch the webpage.")
        return

    soup = BeautifulSoup(response.content, "html.parser")

    # Get commit message
    commit_message = soup.find("div", class_="commit-desc").get_text().strip()
    #print(f"Commit message: {commit_message}")
    
    # Get code changes
    diff_elements = soup.find_all("table", class_="highlight tab-size js-file-line-container")
    for diff_element in diff_elements:
        diff_code = diff_element.get_text()
        print(diff_code)
    return commit_message

commit_url = "https://github.com/CGAL/cgal/commit/28a9cb150ae9b11f9bb37d972be990d87b05cbcf"
get_github_commit_info(commit_url)

'for facets vertices\nthis should be a lot more efficient'

'for facets vertices\nthis should be a lot more efficient'

# Open ai api example

In [21]:
openai.api_key = openai_key
catagories = ""
for i in catogary_lsit_RQ2:
    catagories += i + ", "
#print(catagories)

"""
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages = [
      {"role": "system", "content": f"You are a highly intelligent software high performance enginiering assistant."+ f"when given a git commit messene about hpc you will classify it int to one of these catagories: {catagories}."},
      {"role": "user", "content": f"what is this {get_github_commit_info(commit_url)} git commit messene catagories to ?  "} ,       
    ],   
  
)
print(response['choices'][0]['message']["content"])
"""

{
  "content": "I'm not sure what you mean by \"facets vertices\", could you please provide more context or clarify your question?\n\nAs for more efficient Git commit message categories, the ones I provided earlier are fairly comprehensive and cover a wide range of topics relevant to high performance computing. However, if you have specific categories in mind that you think would be more efficient or useful for your workflow, I'd be happy to discuss them with you.",
  "role": "assistant"
}
